In [806]:
import os
import torch
from torch import nn
import pandas as pd
import numpy as np
import random
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader


DEVICE = torch.device("cpu")
BATCHSIZE = 300
DIR = os.getcwd()
EPOCHS = 15
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = 300 * 100
N_VALID_EXAMPLES = 300 * 50

In [192]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [807]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [496]:
import random

In [796]:
set_random_seed(3407) 
random.randint(0,2)

0

In [808]:
def Get_activation(trial): 
    Activation_names = ['ReLU', 'ReLU6', 'Sigmoid']
    a = random.randint(0,2)
    if a == 0: 
        activation = nn.ReLU()
    elif a == 1: 
        activation = nn.ReLU6()
    elif a == 2:
        activation = nn.Sigmoid()
    print(a)
    
    return activation

In [809]:
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 2, 4)
    layers = []

    in_features = 90
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 20, 150)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(num_features=out_features))
        #activation = trial.suggest_categorical("activation", [nn.ReLU(), nn.ReLU6(), nn.Sigmoid()])
        layers.append(Get_activation(trial))
        #p = trial.suggest_float("dropout_l{}".format(i), 0, 0.1)
        #layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [810]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        super().__init__() 
        self.X = X
        self.y = y
        self.len = len(X)

    def __len__(self): 
        return self.len

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [811]:
def get_mnist():
    train_dl = DataLoader(
    MyDataset(X_train, y_train),
    batch_size = 128,
    shuffle = True
    )
    test_dl = DataLoader(
        MyDataset(X_test, y_test),
        batch_size = 300,
        shuffle = False
    )


    return train_dl, test_dl

In [812]:
loss_fn = nn.MSELoss()

In [813]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [819]:
1e-3

0.001

In [814]:
GAMMA = 0.9995 
1e-3 

0.001

In [815]:
def objective(trial):

    set_random_seed(3407) 
    model = define_model(trial).to(DEVICE)
    optimizer_name = trial.suggest_categorical("optimizer", ["AdamW"])
    lr = trial.suggest_float("lr", 1e-3, 1e-2)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)
    train_loader, valid_loader = get_mnist()
    print('start'*10)
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data.float())
            loss = loss_fn(output.float(), target.float())
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data.float()).view(-1)
                
                correct += (torch.square((output.float() - target.float())).sum()).item()
        accuracy = (correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)) ** (1/2)
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


In [816]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=300, timeout=1000)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-23 19:22:57,857] A new study created in memory with name: no-name-e2ce2b98-4368-4aa6-aa3c-b2cdac61ce01
C:\Users\vladi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


2
1
1
0
startstartstartstartstartstartstartstartstartstart


[I 2021-10-23 19:23:12,258] Trial 0 finished with value: 28.291967113034044 and parameters: {'n_layers': 4, 'n_units_l0': 109, 'n_units_l1': 108, 'n_units_l2': 53, 'n_units_l3': 110, 'optimizer': 'AdamW', 'lr': 0.00536931134815788}. Best is trial 0 with value: 28.291967113034044.
C:\Users\vladi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1
2
2
0
startstartstartstartstartstartstartstartstartstart


[I 2021-10-23 19:23:21,957] Trial 1 finished with value: 16.283335455049333 and parameters: {'n_layers': 4, 'n_units_l0': 55, 'n_units_l1': 29, 'n_units_l2': 85, 'n_units_l3': 108, 'optimizer': 'AdamW', 'lr': 0.007890493619812288}. Best is trial 1 with value: 16.283335455049333.
C:\Users\vladi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

0
0
1
0
startstartstartstartstartstartstartstartstartstart


[I 2021-10-23 19:36:12,724] Trial 298 pruned. 
C:\Users\vladi\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1
1
0
0
startstartstartstartstartstartstartstartstartstart


[I 2021-10-23 19:36:13,439] Trial 299 pruned. 


Study statistics: 
  Number of finished trials:  300
  Number of pruned trials:  276
  Number of complete trials:  24
Best trial:
  Value:  12.126558098987735
  Params: 
    n_layers: 4
    n_units_l0: 27
    n_units_l1: 133
    n_units_l2: 51
    n_units_l3: 149
    optimizer: AdamW
    lr: 0.00769958872312551
